In [1]:
import pandas as pd
import numpy as np
import pickle
import string
import nltk
import nltk
import re
from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from nltk import word_tokenize
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д','а','и','\n'])

In [2]:
df_res = pd.read_csv(r'C:/Users/Jora/Desktop/trademarks-2022/smalldata.csv',sep=';',encoding='ANSI')

In [3]:

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('russian')
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text, flags=re.I)

mystem = Mystem() 

russian_stopwords = nltk.corpus.stopwords.words('russian')
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д','а','и','\n'])
def lemmatize_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
    text = " ".join(tokens)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jora\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)

In [5]:
tech_text = "радиоактивные вещества "

In [6]:
tech_text = remove_multiple_spaces(remove_numbers(remove_punctuation(tech_text.lower())))
tech_text = remove_stop_words(tech_text)
tech_text = lemmatize_text(tech_text)
tech_text

'радиоактивный вещество'

In [7]:

with open('tnved_class.pkl', 'rb') as fid:
    logreg = pickle.load(fid)

In [8]:
my_tags=df_res['TNVED'].unique()
tech_pred = logreg.predict_proba([tech_text])
res=pd.DataFrame([np.round(tech_pred[0],4),(my_tags).astype(float)])
res

,0,1,2,3,4,5,6,7,8,9,...,283,284,285,286,287,288,289,290,291,292
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0001,0.0,0.9593,0.0,0.0,0.0,0.0,0.0001,0.0,0.0001
1,101.0,102.0,103.0,104.0,105.0,106.0,201.0,202.0,203.0,204.0,...,2842.0000,2843.0,2844.0000,2845.0,2846.0,2847.0,2849.0,2850.0000,2852.0,2853.0000


In [9]:
zeros = []
ones = []
for index, row in res.iterrows():
    if index == 0:
        maxes = np.sort(row)[len(row)-5:]
        
for column in res.columns:
    if res[column][0] in maxes:
        print(res[column])

0       0.0025
1    1702.0000
Name: 136, dtype: float64
0       0.0007
1    1802.0000
Name: 140, dtype: float64
0       0.0293
1    2101.0000
Name: 159, dtype: float64
0       0.0007
1    2502.0000
Name: 185, dtype: float64
0       0.0015
1    2828.0000
Name: 270, dtype: float64
0       0.9593
1    2844.0000
Name: 285, dtype: float64


In [10]:
data_res=pd.DataFrame()
data_res['class']=my_tags
data_res['predict']=np.round(tech_pred[0],4)
data_res.sort_values(by='predict',ascending=False)

,class,predict
285,2844,0.9593
159,2101,0.0293
136,1702,0.0025
270,2828,0.0015
140,1802,0.0007
...,...,...
126,1517,0.0000
127,1518,0.0000
128,1520,0.0000
129,1521,0.0000
